In [31]:
!pip install pyTDC

In [32]:
#To obtain the label names,
from tdc.utils import retrieve_label_name_list
label_list = retrieve_label_name_list('Leenay')

In [33]:
#Then, proceed with the usual data loader
from tdc.single_pred import CRISPROutcome
data = CRISPROutcome(name = 'Leenay', label_name = label_list[0])
df = data.get_split()
print(df)

Found local copy...
Loading...
Done!


{'train':         GuideSeq_ID                 GuideSeq         Y
0        GuideSeq 1  CTGCAGGGCTAGTTTCCTATAGG  0.069572
1        GuideSeq 2  GAGATGCGGACCACCCAGCTGGG  0.287647
2        GuideSeq 3  GCAAACGGAAGTGCAATTGTCGG  0.137004
3        GuideSeq 4  GTCATCGCTGAGTTGAGGAAGGG  0.093889
4        GuideSeq 5  ATATGATTATCCCTGCACAAGGG  0.526525
...             ...                      ...       ...
1060  GuideSeq 1517  ATGCCCGACCAAAGACAACCAGG  0.093236
1061  GuideSeq 1518  CACGCTGTCATCCACCAGGTAGG  0.069054
1062  GuideSeq 1519  GGGCTCCAGAGTCTGATACAGGG  0.421439
1063  GuideSeq 1520  AAAGACTTCGGTCCTCTAGTAGG  0.206277
1064  GuideSeq 1521  TCGTCGCCCAGTGTCCAAGTGGG  0.284117

[1065 rows x 3 columns], 'valid':        GuideSeq_ID                 GuideSeq         Y
0     GuideSeq 841  CCTTGTGAATCTCCTCACGGAGG  0.394674
1     GuideSeq 101  CGCTTTACCTGCGACCAGGTAGG  0.072320
2     GuideSeq 235  CATTCTCCAAGTGACAAGGTAGG  0.049883
3     GuideSeq 162  CTTGGTCATGCCTTCCCGGTAGG  0.206285
4     GuideSeq 421  AAAGT

In [34]:
import pandas as pd
# Convert the dictionary to a DataFrame
df = pd.DataFrame(df['train'])

# Write the DataFrame to a CSV file
df.to_csv('train.csv', index=False)

print('DataFrame converted and saved to CSV successfully.')
print(df)

DataFrame converted and saved to CSV successfully.
        GuideSeq_ID                 GuideSeq         Y
0        GuideSeq 1  CTGCAGGGCTAGTTTCCTATAGG  0.069572
1        GuideSeq 2  GAGATGCGGACCACCCAGCTGGG  0.287647
2        GuideSeq 3  GCAAACGGAAGTGCAATTGTCGG  0.137004
3        GuideSeq 4  GTCATCGCTGAGTTGAGGAAGGG  0.093889
4        GuideSeq 5  ATATGATTATCCCTGCACAAGGG  0.526525
...             ...                      ...       ...
1060  GuideSeq 1517  ATGCCCGACCAAAGACAACCAGG  0.093236
1061  GuideSeq 1518  CACGCTGTCATCCACCAGGTAGG  0.069054
1062  GuideSeq 1519  GGGCTCCAGAGTCTGATACAGGG  0.421439
1063  GuideSeq 1520  AAAGACTTCGGTCCTCTAGTAGG  0.206277
1064  GuideSeq 1521  TCGTCGCCCAGTGTCCAAGTGGG  0.284117

[1065 rows x 3 columns]


In [35]:
!pip install xgboost


## Importing the Libraries 

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Assuming df is your DataFrame with columns 'GuideSeq_ID', 'GuideSeq', and 'Y'

train_df = pd.read_csv("/workspace/Crispr-repair-outcome/train.csv")

train_GuideSeq = train_df['GuideSeq']
train_Y = train_df['Y']

## Encoding 

In [37]:
# Encode GuideSeq column if needed (assuming it's categorical)
label_encoder = LabelEncoder()
train_df['GuideSeq_encoded'] = label_encoder.fit_transform(train_GuideSeq)

##  Data Spltting

In [38]:
# Split the data into features (X) and target variable (y)
X = train_df[['GuideSeq_encoded']]  # Use appropriate features
y = train_df['Y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Buidling

In [39]:
# Models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'SVR': SVR(),
    'XGBoost': XGBRegressor()
}

## Model Prediction and Evaluation

In [40]:
for model_name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Make predictionsmse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Model: {model_name}')
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r2}')

Model: Linear Regression
Mean Squared Error: 0.03554477528886801
R-squared: 0.00048572515817491
Model: Random Forest
Mean Squared Error: 0.03554477528886801
R-squared: -0.5491099875910446
Model: Gradient Boosting
Mean Squared Error: 0.03554477528886801
R-squared: -0.0738030269140022
Model: SVR
Mean Squared Error: 0.03554477528886801
R-squared: -0.006220921433394144
Model: XGBoost
Mean Squared Error: 0.03554477528886801
R-squared: -0.34872538012483445


## Save the Model to a Pickle File

In [44]:
import pickle

best_model = None
best_r2 = float('-inf')  # Initialize with negative infinity

# Check if the current model has a higher R-squared than the best so far
if r2 > best_r2:
    best_r2 = r2
    best_model = model


# Save the best model to a file using pickle
if best_model is not None:
    with open('regression.pkl', 'wb') as file:
        pickle.dump(best_model, file)
        print('Best model saved to best_model.pkl')
else:
    print('No best model found.')

Best model saved to best_model.pkl
